In [1]:
from importlib.metadata import version
import os

try:
    import xradio

    print("XRADIO version", version("xradio"), "already installed.")
except ImportError as e:
    print(e)
    print("Installing XRADIO")

    os.system("pip install xradio")

    import xradio

    print("xradio version", version("xradio"), " installed.")

from xradio.data.datasets import download

# These are the three API functions that are currently supported
from xradio.image import load_image, read_image, write_image

XRADIO version 0.0.12 already installed.


## Download dataset

https://open-bitbucket.nrao.edu/projects/CASA/repos/casatestdata/browse/unittest/tclean/venus_ephem_test.ms

In [2]:
from xradio.data.datasets import download

ms_file = "sdimaging.ms"
download(file=ms_file, source="dropbox")

url https://www.dropbox.com/scl/fi/6ng24m36yv5j0k25ybynw/sdimaging.ms.zip?rlkey=b4uokqafyfaruwzkr6jl98mir&dl


./sdimaging.ms.zip: 100%|██████████| 28.3M/28.3M [00:00<00:00, 37.5MiB/s]


## Convert to Processing Set

In [3]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set

partition_scheme = "ddi_intent_field"
# partition_scheme="ddi_state"

main_chunksize = {"time": 63}  # baseline, polarization
outfile = "sdimaging.vis.zarr"
convert_msv2_to_processing_set(
    in_file=ms_file,
    out_file=outfile,
    partition_scheme=partition_scheme,
    parallel=False,
    overwrite=True,
    main_chunksize=main_chunksize,
)

DDI 0, STATE [0], FIELD 0
tb_tool 3843
DDI 0, STATE [1], FIELD 0
tb_tool 0
DDI 1, STATE [0], FIELD 0
tb_tool 0
DDI 1, STATE [1], FIELD 0
tb_tool 0


## Processing Set

In [4]:
ps_name = outfile

from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set(ps_name=outfile)
ps.summary()

,name,ddi,intent,field_id,field_name,start_frequency,end_frequency
0,sdimaging_ddi_0_intent_OBSERVE_TARGET_ON_SOURC...,0,"OBSERVE_TARGET_ON_SOURCE,FREQUENCY_SWITCH_SIG",0,FLS3a,1.419395e+09,1.421893e+09


In [5]:
ps.keys()

dict_keys(['sdimaging_ddi_0_intent_OBSERVE_TARGET_ON_SOURCE,FREQUENCY_SWITCH_SIG_field_id_0'])

In [6]:
ps['sdimaging_ddi_0_intent_OBSERVE_TARGET_ON_SOURCE,FREQUENCY_SWITCH_SIG_field_id_0']

<xarray.Dataset>
Dimensions:                     (time: 3843, baseline_id: 1, frequency: 1024,
                                 polarization: 2, uvw_label: 3)
Coordinates:
    baseline_antenna1_id        (baseline_id) int64 dask.array<chunksize=(1,), meta=np.ndarray>
    baseline_antenna2_id        (baseline_id) int64 dask.array<chunksize=(1,), meta=np.ndarray>
  * baseline_id                 (baseline_id) int64 0
  * frequency                   (frequency) float64 1.419e+09 ... 1.422e+09
  * polarization                (polarization) <U2 'XX' 'YY'
  * time                        (time) float64 1.069e+09 1.069e+09 ... 1.069e+09
  * uvw_label                   (uvw_label) <U1 'u' 'v' 'w'
Data variables:
    EFFECTIVE_INTEGRATION_TIME  (time, baseline_id) float64 dask.array<chunksize=(63, 1), meta=np.ndarray>
    FLAG                        (time, baseline_id, frequency, polarization) bool dask.array<chunksize=(63, 1, 1024, 2), meta=np.ndarray>
    SPECTRUM                    (time, baseline_id, frequency, polarization) float32 dask.array<chunksize=(63, 1, 1024, 2), meta=np.ndarray>
    TIME_CENTROID               (time, baseline_id) float64 dask.array<chunksize=(63, 1), meta=np.ndarray>
    UVW                         (time, baseline_id, uvw_label) float64 dask.array<chunksize=(63, 1, 3), meta=np.ndarray>
    WEIGHT                      (time, baseline_id, frequency, polarization) float32 dask.array<chunksize=(63, 1, 1024, 2), meta=np.ndarray>
Attributes:
    data_groups:  {'base': {'flag': 'FLAG', 'uvw': 'UVW', 'visibility': 'VISI...
    ddi:          0
    field_info:   {'code': '', 'delay_direction': {'attrs': {'frame': 'FK5', ...
    intent:       OBSERVE_TARGET_ON_SOURCE,FREQUENCY_SWITCH_SIG
    antenna_xds:  <xarray.Dataset>\nDimensions:        (antenna_id: 1, xyz_la...

In [7]:
ps['sdimaging_ddi_0_intent_OBSERVE_TARGET_ON_SOURCE,FREQUENCY_SWITCH_SIG_field_id_0'].attrs['antenna_xds']

<xarray.Dataset>
Dimensions:        (antenna_id: 1, xyz_label: 3)
Coordinates:
  * antenna_id     (antenna_id) int64 0
    mount          (antenna_id) <U6 dask.array<chunksize=(1,), meta=np.ndarray>
    name           (antenna_id) <U3 dask.array<chunksize=(1,), meta=np.ndarray>
    station        (antenna_id) <U9 dask.array<chunksize=(1,), meta=np.ndarray>
    type           (antenna_id) <U1 dask.array<chunksize=(1,), meta=np.ndarray>
  * xyz_label      (xyz_label) <U1 'x' 'y' 'z'
Data variables:
    DISH_DIAMETER  (antenna_id) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    FEED_OFFSET    (antenna_id, xyz_label) float64 dask.array<chunksize=(1, 3), meta=np.ndarray>
    POSITION       (antenna_id, xyz_label) float64 dask.array<chunksize=(1, 3), meta=np.ndarray>